In [1]:
# Import libraries


# Homework

In this homework, we will use Bank credit scoring dataset from [here](https://www.kaggle.com/datasets/kapturovalexander/bank-credit-scoring/data) aboyt data.
While about environment, we will use __Pipenv__.

## Question 1
- Install Pipenv
- What's the version of pipenv you installed?
- Use --version to find out

To install Pipenv let's execute the following command by terminal:
<code>
    pip install --user pipenv
</code>
Then to check the version of pipenv let's run the following command:
<code>
    pipenv --version
</code>
In my case the pipenv version is

In [2]:
# Print pipenv version
!pipenv --version

pipenv, version 2023.10.3


## Question 2
- Use Pipenv to install Scikit-Learn version 1.3.1
- What's the first hash for scikit-learn you get in Pipfile.lock?

To create the environment and install scikit-learn, let's execute the following command into the current folder:
<code>
    pipenv install scikit-learn==1.3.1
</code>
The files Pipfile and Pipfile.lock will be created. 
After that, to activate its, let's run:
<code>
    pipenv shell
</code>
Let's activate Jupyter lab by using the following command:
<code>
    jupyter lab
</code>

In [3]:
# Let's show Pipfile.lock
!cat Pipfile.lock

{
    "_meta": {
        "hash": {
            "sha256": "1ad0d328b3b81fc037866d3a2948adb16c827a0792311e7a69f5172f4bcafb82"
        },
        "pipfile-spec": 6,
        "requires": {
            "python_version": "3.9"
        },
        "sources": [
            {
                "name": "pypi",
                "url": "https://pypi.org/simple",
                "verify_ssl": true
            }
        ]
    },
    "default": {
        "blinker": {
            "hashes": [
                "sha256:152090d27c1c5c722ee7e48504b02d76502811ce02e1523553b4cf8c8b3d3a8d",
                "sha256:296320d6c28b006eb5e32d4712202dbcdcbf5dc482da298c2f44881c43884aaa"
            ],
            "markers": "python_version >= '3.7'",
            "version": "==1.6.3"
        },
        "click": {
            "hashes": [
                "sha256:ae74fb96c20a0277a1d615f1e4d73c8414f5a98db8b799a7931d1582f3390c28",
                "sha256:ca9853ad459e787e2192211578cc907e7594e294c7ccc834310722b41b9ca6de"
          

In [4]:
import json

In [5]:
#  Let's show the first hash for scikit-learn you get in Pipfile.lock
with open('Pipfile.lock','r') as f:
    data = json.load(f)
    print(data['default']['scikit-learn']['hashes'][0])

sha256:0c275a06c5190c5ce00af0acbb61c06374087949f643ef32d355ece12c4db043


## Models

We've prepared a dictionary vectorizer and a model.

They were trained (roughly) using this code:
<code>
features = ['job','duration', 'poutcome']
dicts = df[features].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
X = dv.fit_transform(dicts)
model = LogisticRegression().fit(X, y)
</code>

And then saved with Pickle. Download them:
<code>
PREFIX=https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2023/05-deployment/homework
wget $PREFIX/model1.bin
wget $PREFIX/dv.bin
</code>

## Question 3

Let's use these models!
- Write a script for loading these models with pickle
- Score this client:
<code>
{"job": "retired", "duration": 445, "poutcome": "success"}
</code>
What's the probability that this client will get a credit?
- 0.162
- 0.392
- 0.652
- 0.902

In [6]:
# Import pickle module
import pickle

In [7]:
# Loading model and dict vector
file_dv = 'dv.bin'
with open(file_dv,'rb') as f:
    dv = pickle.load(f)

file_model = 'model1.bin'
with open(file_model,'rb') as f:
    model = pickle.load(f)

/home/antoniopeduto/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.3.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/antoniopeduto/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.3.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
!md5sum model1.bin dv.bin

8ebfdf20010cfc7f545c43e3b52fc8a1  model1.bin
924b496a89148b422c74a62dbc92a4fb  dv.bin


In [9]:
model.classes_

array(['no', 'yes'], dtype=object)

In [10]:
x_sample_dict = {"job": "retired", "duration": 445, "poutcome": "success"}
x_sample = dv.transform(x_sample_dict)
x_sample.shape

(1, 17)

In [11]:
y_sample_pred = model.predict_proba(x_sample)[0,1]
y_sample_pred

0.9019309332297606

## Question 4

Now let's serve this model as a web service
- Install Flask and gunicorn (or waitress, if you're on Windows)
- Write Flask code for serving the model
- Now score this client using requests:
<code>
url = "YOUR_URL"
client = {"job": "unknown", "duration": 270, "poutcome": "failure"}
requests.post(url, json=client).json()
</code>
What's the probability that this client will get a credit?
- 0.140
- 0.440
- 0.645
- 0.845

In [12]:
# Install flask
!pipenv install flask 

Installing flask...
Resolving flask...
✔ Installation Succeeded
⠋ Installing flask...
Installing dependencies from Pipfile.lock (cafb82)...


In [13]:
# Install gunicorn
!pipenv install gunicorn

Installing gunicorn...
Resolving gunicorn...
✔ Installation Succeeded
⠋ Installing gunicorn...
Installing dependencies from Pipfile.lock (cafb82)...


Let's open terminal and run the command
<code>
    gunicorn --bind 0.0.0.0:9696 predict:app
</code>

In [14]:
import requests

In [15]:
url = "http://localhost:9696/predict"
client = {"job": "unknown", "duration": 270, "poutcome": "failure"}
response = requests.post(url=url, json=client).json()
response

{'prob_get_credit': 0.13968947052356817}

## Docker

Install Docker. We will use it for the next two questions.

For these questions, we prepared a base image: svizor/zoomcamp-model:3.10.12-slim. You'll need to use it (see Question 5 for an example).

This image is based on python:3.10.12-slim and has a logistic regression model (a different one) as well a dictionary vectorizer inside.

This is how the Dockerfile for this image looks like:

<code>
FROM python:3.10.12-slim
WORKDIR /app
COPY ["model2.bin", "dv.bin", "./"]
</code>

We already built it and then pushed it to svizor/zoomcamp-model:3.10.12-slim.

## Question 5

Download the base image svizor/zoomcamp-model:3.10.12-slim. You can easily make it by using docker pull command.

So what's the size of this base image?
- 47 MB
- 147 MB
- 374 MB
- 574 MB

You can get this information when running docker images - it'll be in the "SIZE" column.

Let's download image by executing in terminal the command:
<code>
    docker pull svizor/zoomcamp-model:3.10.12-slim
</code>
and then show the size:
<code>
    docker images | grep svizor
</code>
The size is 147 MB.

## Dockerfile

Now create your own Dockerfile based on the image we prepared.

It should start like that:

FROM svizor/zoomcamp-model:3.10.12-slim

--add your stuff here

Now complete it:

- Install all the dependencies from the Pipenv file
- Copy your Flask script
- Run it with Gunicorn

After that, you can build your docker image.

# Question 6

Let's run your docker container!

After running it, score this client once again:

<code>
url = "YOUR_URL"
client = {"job": "retired", "duration": 445, "poutcome": "success"}
requests.post(url, json=client).json()
</code>
 
 
What's the probability that this client will get a credit now?
- 0.168
- 0.530
- 0.730
- 0.968

Let's write the following Dockerfile:
<code>
FROM svizor/zoomcamp-model:3.10.12-slim
RUN pip install pipenv
WORKDIR /app
COPY ["Pipfile","Pipfile.lock","./"]
RUN pipenv install --system --deploy
COPY ["predict.py","model1.bin","./"]
EXPOSE 9696
ENTRYPOINT ["gunicorn", "--bind=0.0.0.0:9696","predict:app"]
</code>

Then let's build the image by the following command:
<code>
    sudo docker build -t zoomcampapp ./
</code>

And let's runt it
<code>
    sudo docker run -it --rm -p 9696:9696 zoomcampapp
</code>
Finally let's test such app

In [18]:
url = "http://localhost:9696/predict"
client={"job": "retired", "duration": 445, "poutcome": "success"}
response = requests.post(url=url, json=client).json()
response

{'prob_get_credit': 0.9019309332297606}